In [1]:
import json

from promptify.models.nlp.openai_model import OpenAI
from promptify.prompts.nlp.prompter import Prompter

In [2]:
model = OpenAI(api_key="")
nlp_prompter = Prompter(model)

In [3]:
data = json.load(open("data/sql.json",'r'))

In [4]:
len(data)

10

In [5]:
examples = []
for sample in data[:5]:
    print(sample,"\n")
    examples.append((sample['sentence'],sample['query']))

{'sentence': 'Retrieve the details of all customers.', 'query': 'SELECT * FROM customers;'} 

{'sentence': 'Show the total number of orders placed by each customer.', 'query': 'SELECT customer_id, COUNT(*) as num_orders FROM orders GROUP BY customer_id;'} 

{'sentence': 'Display the total revenue generated by each product category.', 'query': 'SELECT product_name, SUM(price * quantity) as revenue FROM orders GROUP BY product_name;'} 

{'sentence': 'Retrieve the list of all orders placed by a specific customer.', 'query': 'SELECT * FROM orders WHERE customer_id = 123;'} 

{'sentence': 'Get the details of all customers whose last name starts with "S".', 'query': "SELECT * FROM customers WHERE last_name LIKE 'S%';"} 



In [6]:
example_schema = {
    "customers": {
        "customer_id": int,
        "first_name": str,
        "last_name": str,
        "email": str,
        "phone": str,
        "address": str
    },
    "orders": {
        "order_id": int,
        "customer_id": int,
        "order_date": str,
        "product_name": str,
        "price": float,
        "quantity": int
    }
}


In [7]:
schema ={
    "cars":{
        "car_id":int,
        "car_name":str,
        "car_price":float,
        "car_color":str,
        "car_model":str,
        "car_year":int
    }
}

In [9]:
prompt = nlp_prompter.generate_prompt('sql_writer.jinja',
                                      examples=examples,
                                        example_schema=example_schema,
                                        schema = schema,
                                      text_input="Write a SQL query to get car names and prices from the cars table where the car color is red and the car price is greater than 10000.",
                                     description="SQL Writer")
print(prompt)

SQL Writer
You are a highly intelligent and accurate SQL query creator. You take a sentence and turn it into a SQL query. 
Sometimes you are also provided with a table and you have to create a query that returns the correct answer.
Your output format is a dictionary with a single key 'Q' and the value is the SQL query, so [{'Q':Query}] form, no other form.{'customers': {'customer_id': <class 'int'>, 'first_name': <class 'str'>, 'last_name': <class 'str'>, 'email': <class 'str'>, 'phone': <class 'str'>, 'address': <class 'str'>}, 'orders': {'order_id': <class 'int'>, 'customer_id': <class 'int'>, 'order_date': <class 'str'>, 'product_name': <class 'str'>, 'price': <class 'float'>, 'quantity': <class 'int'>}}
Examples:

Input: Retrieve the details of all customers.
Output: [{'Q': 'SELECT * FROM customers;' }]

Input: Show the total number of orders placed by each customer.
Output: [{'Q': 'SELECT customer_id, COUNT(*) as num_orders FROM orders GROUP BY customer_id;' }]

Input: Display the

In [10]:
output = nlp_prompter.fit('sql_writer.jinja',
                                      examples=examples,
                                        example_schema=example_schema,
                                        schema = schema,
                                      text_input="Write a SQL query to get car names and prices from the cars table where\
                                      the car color is red and the car price is greater than 10000.",
                                     description="SQL Writer",
                                     model_name="text-davinci-003")

In [11]:
output

{'prompt_tokens': 558,
 'completion_tokens': 34,
 'total_tokens': 592,
 'text': " [{'Q': 'SELECT car_name, car_price FROM cars WHERE car_color = 'red' AND car_price > 10000;' }]"}